<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n20_x4_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n20_x4_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...          y_hat          e_hat
rep  index                        ...                              
0.0  0.0    11.590788  16.786837  ...    4655.338328   -8453.725222
     1.0    19.473311  28.651164  ...  -61331.639386  150835.923522
     2.0    15.493246  17.931694  ...  -84965.364757   95236.876379
     3.0    25.402813  18.849997  ...   68124.425483 -622582.536310
     4.0    20.891068  36.258961  ...  -36004.077621  -63521.129447
...               ...        ...  ...            ...            ...
99.0 15.0   10.596038  33.165883  ...     767.090440  -13119.397351
     16.0   34.526054  31.567667  ...  103189.824177  666727.252417
     17.0   23.641074  47.053112  ...  -16582.783287   94592.772149
     18.0   49.062809  40.757730  ...  146895.258390  813694.516771
     19.0   12.747758  46.180233  ...    -606.658394  -11853.747747

[2000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...          y_hat          e_hat
rep  index                        ...                              
0.0  0.0    11.590788  16.786837  ...    4655.338328   -8453.725222
     1.0    19.473311  28.651164  ...  -61331.639386  150835.923522
     2.0    15.493246  17.931694  ...  -84965.364757   95236.876379
     3.0    25.402813  18.849997  ...   68124.425483 -622582.536310
     4.0    20.891068  36.258961  ...  -36004.077621  -63521.129447
...               ...        ...  ...            ...            ...
99.0 15.0   10.596038  33.165883  ...     767.090440  -13119.397351
     16.0   34.526054  31.567667  ...  103189.824177  666727.252417
     17.0   23.641074  47.053112  ...  -16582.783287   94592.772149
     18.0   49.062809  40.757730  ...  146895.258390  813694.516771
     19.0   12.747758  46.180233  ...    -606.658394  -11853.747747

[2000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,11.590788,16.786837,12.389072,-3948.470680,-3798.386894,4655.338328,-8453.725222
1,0.0,1.0,19.473311,28.651164,47.644203,89228.255662,89504.284136,-61331.639386,150835.923522
2,0.0,2.0,15.493246,17.931694,47.173341,10046.474132,10271.511622,-84965.364757,95236.876379
3,0.0,3.0,25.402813,18.849997,15.346461,-554751.335415,-554458.110827,68124.425483,-622582.536310
4,0.0,4.0,20.891068,36.258961,42.973384,-99818.350091,-99525.207068,-36004.077621,-63521.129447
...,...,...,...,...,...,...,...,...,...
1995,99.0,15.0,10.596038,33.165883,14.086414,-12510.519589,-12352.306911,767.090440,-13119.397351
1996,99.0,16.0,34.526054,31.567667,20.920122,769514.328268,769917.076594,103189.824177,666727.252417
1997,99.0,17.0,23.641074,47.053112,42.760558,77678.764454,78009.988862,-16582.783287,94592.772149
1998,99.0,18.0,49.062809,40.757730,32.209242,960021.180099,960589.775161,146895.258390,813694.516771


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...          y_hat          e_hat
rep  index                        ...                              
0.0  0.0    11.590788  16.786837  ...    4655.338328   -8453.725222
     1.0    19.473311  28.651164  ...  -61331.639386  150835.923522
     2.0    15.493246  17.931694  ...  -84965.364757   95236.876379
     3.0    25.402813  18.849997  ...   68124.425483 -622582.536310
     4.0    20.891068  36.258961  ...  -36004.077621  -63521.129447
...               ...        ...  ...            ...            ...
99.0 15.0   10.596038  33.165883  ...     767.090440  -13119.397351
     16.0   34.526054  31.567667  ...  103189.824177  666727.252417
     17.0   23.641074  47.053112  ...  -16582.783287   94592.772149
     18.0   49.062809  40.757730  ...  146895.258390  813694.516771
     19.0   12.747758  46.180233  ...    -606.658394  -11853.747747

[2000 rows x 7 columns]

In [7]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    # คำนวณค่าสถิติ
    GQ_test_clusters = MSE1 / MSE0

    df1 = DF1
    df0 = DF0

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [8]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
3.0,8.0,4.0,22875974250115.17,56978615636688.09,2859496781264.3965,14244653909172.023,4.981524722288165,0.05191273632549742,0.9740436318372513,Reject005=1 : Homoscedasticity
4.0,5.0,7.0,33722151089512.15,81696068995078.42,6744430217902.43,11670866999296.918,1.7304452151225087,0.5650307012226534,0.7174846493886733,Reject005=1 : Homoscedasticity
5.0,8.0,4.0,524373600776.9841,69080022859409.88,65546700097.12301,17270005714852.47,263.4763563880844,3.2017780204895985e-08,0.9999999839911099,Reject005=0 : Heteroscedasticity
11.0,6.0,6.0,10966678826141.023,81737531428722.39,1827779804356.8372,13622921904787.064,7.4532620791161035,0.027512625671992197,0.9862436871640039,Reject005=0 : Heteroscedasticity
12.0,6.0,6.0,10809906675425.78,13093175096551.26,1801651112570.9631,2182195849425.21,1.2112199938151222,0.8219828853921345,0.5890085573039328,Reject005=1 : Homoscedasticity
13.0,9.0,3.0,1014436606935.0387,39291317248696.89,112715178548.33763,13097105749565.63,116.19646899595665,3.259498855268106e-07,0.9999998370250572,Reject005=0 : Heteroscedasticity
17.0,4.0,8.0,641947719822.304,33237562603846.656,160486929955.576,4154695325480.832,25.888060333828328,0.006912241795801188,0.9965438791020994,Reject005=0 : Heteroscedasticity
19.0,3.0,9.0,1852502992190.179,12981982663369.875,617500997396.7263,1442442518152.2083,2.3359355276077087,0.5243955075876772,0.7378022462061614,Reject005=1 : Homoscedasticity
23.0,6.0,6.0,2502294138442.9204,120024673772731.52,417049023073.82007,20004112295455.254,47.96585338581266,0.00016517719145259235,0.9999174114042737,Reject005=0 : Heteroscedasticity


In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,7.0,5.0,23932547183613.543,1580519270093.7861,3418935311944.792,316103854018.7572,0.09245681043287945,0.018542500435099606,Reject005=0 : Heteroscedasticity
1.0,6.0,6.0,27944024435107.375,3381691913070.276,4657337405851.2295,563615318845.046,0.12101663884968908,0.02126269767425046,Reject005=0 : Heteroscedasticity
2.0,1.0,11.0,219905135121.2712,1769050588361.1292,219905135121.2712,160822780760.10266,0.7313279913696132,0.5339561886767572,Reject005=1 : Homoscedasticity
3.0,8.0,4.0,22875974250115.17,56978615636688.09,2859496781264.3965,14244653909172.023,4.981524722288165,0.05191273632549742,Reject005=1 : Homoscedasticity
4.0,5.0,7.0,33722151089512.15,81696068995078.42,6744430217902.43,11670866999296.918,1.7304452151225087,1.4349692987773466,Reject005=1 : Homoscedasticity
5.0,8.0,4.0,524373600776.9841,69080022859409.88,65546700097.12301,17270005714852.47,263.4763563880844,3.2017780204895985e-08,Reject005=0 : Heteroscedasticity
6.0,3.0,9.0,13095137279780.182,3075496223755.012,4365045759926.727,341721802639.4458,0.07828596111789263,0.00271247566755043,Reject005=0 : Heteroscedasticity
7.0,4.0,8.0,13773050928148.406,9752380790289.44,3443262732037.1016,1219047598786.18,0.35403850755965044,0.1973806453971252,Reject005=1 : Homoscedasticity
8.0,7.0,5.0,39449941878796.97,11604090727081.72,5635705982685.281,2320818145416.344,0.4118061077967252,0.3458571085170602,Reject005=1 : Homoscedasticity


In [10]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    63
Reject005=1 : Homoscedasticity      37
Name: Result_test, dtype: int64